<a href="https://colab.research.google.com/github/ahsank/StockML/blob/main/Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Backtesting.py_ Quick Start User Guide
=======================

It uses *backtesting.py* Python framework for [backtesting](https://www.investopedia.com/terms/b/backtesting.asp) trading strategies. See [Quickstart](https://github.com/kernc/backtesting.py/blob/master/doc/examples/Quick%20Start%20User%20Guide.ipynb)


## Data
DataFrame should ideally be indexed with a _datetime index_ (convert it with [`pd.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html));
otherwise a simple range index will do.

In [ ]:
!pip install backtesting

In [ ]:
!pip install yahoo_fin

In [99]:
from yahoo_fin import stock_info
ticker = 'ARKK'
arkk = stock_info.get_data('ARKK')
spy = stock_info.get_data('SPY')


In [100]:
arkk.columns = map(str.title, df.columns)
spy.columns = map(str.title, df.columns)
# df.Close = df.Adjclose
# df.drop('Adjclose', axis=1, inplace=True)
spy

,Open,High,Low,Close,Adjclose,Volume,Ticker
1993-01-29,43.968750,43.968750,43.750000,43.937500,24.763748,1003200,SPY
1993-02-01,43.968750,44.250000,43.968750,44.250000,24.939867,480500,SPY
1993-02-02,44.218750,44.375000,44.125000,44.343750,24.992708,201300,SPY
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.256901,529400,SPY
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.362570,531500,SPY
...,...,...,...,...,...,...,...
2024-04-16,504.940002,506.500000,502.209991,503.529999,503.529999,73484000,SPY
2024-04-17,506.049988,506.220001,499.119995,500.549988,500.549988,75910300,SPY
2024-04-18,501.980011,504.130005,498.559998,499.519989,499.519989,74548100,SPY
2024-04-19,499.440002,500.459991,493.859985,495.160004,495.160004,102129100,SPY


In [86]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [87]:
def ToSeries(values):
  return pd.Series(values)

In [88]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [107]:
class AboveSma(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 200

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.Close = self.I(ToSeries, self.data.Close)

    def next(self):
        # If price crosses above sma1 and sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.Close, self.sma1) and crossover(self.Close, self.sma2):
            self.position.close()
            self.buy()

        # Else, if price crosses below sma1 and sma2, close any existing
        # long trades
        elif crossover(self.sma1, self.Close) and crossover(self.sma2, self.Close):
            self.position.close()
            # self.sell()

In [69]:
class CautiousSma(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 200

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.Close = self.I(ToSeries, self.data.Close)

    def next(self):
        # If price crosses above sma1 and sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.Close, self.sma1) and crossover(self.Close, self.sma2):
            self.position.close()
            self.buy()
        # Else, if price crosses below sma1 or sma2, close any existing
        # long trades
        elif crossover(self.sma1, self.Close) or crossover(self.sma2, self.Close):
            self.position.close()
            # self.sell()

## Backtesting

 See
[`Backtest`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest)


In [109]:
from backtesting import Backtest

bt = Backtest(arkk, AboveSma, cash=10_000, commission=0)
stats = bt.run()
stats

Start                     2014-10-31 00:00:00
End                       2024-04-22 00:00:00
Duration                   3461 days 00:00:00
Exposure Time [%]                   53.336131
Equity Final [$]                 49457.755972
Equity Peak [$]                   72518.76111
Return [%]                          394.57756
Buy & Hold Return [%]               109.02846
Return (Ann.) [%]                   18.417158
Volatility (Ann.) [%]               29.568343
Sharpe Ratio                         0.622867
Sortino Ratio                        1.058177
Calmar Ratio                         0.433305
Max. Drawdown [%]                  -42.503897
Avg. Drawdown [%]                   -4.712885
Max. Drawdown Duration     1165 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                   16
Win Rate [%]                            56.25
Best Trade [%]                      152.80528
Worst Trade [%]                     -6.137094
Avg. Trade [%]                    



[`Backtest.plot()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.plot)
method provides the same insights in a more visual form.

In [110]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p4570', ...)

## Optimization

 optimize the two parameters by calling
[`Backtest.optimize()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.optimize)


In [92]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(100, 250, 10),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 114 ms, sys: 28.7 ms, total: 143 ms
Wall time: 3.71 s


Start                     2014-10-31 00:00:00
End                       2024-04-22 00:00:00
Duration                   3461 days 00:00:00
Exposure Time [%]                   65.631557
Equity Final [$]                   71026.5707
Equity Peak [$]                  91340.380186
Return [%]                         610.265707
Buy & Hold Return [%]               109.02846
Return (Ann.) [%]                   23.037345
Volatility (Ann.) [%]                33.97301
Sharpe Ratio                         0.678107
Sortino Ratio                        1.212745
Calmar Ratio                         0.541623
Max. Drawdown [%]                  -42.533928
Avg. Drawdown [%]                   -5.330826
Max. Drawdown Duration      881 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                     487.559834
Worst Trade [%]                      -3.08521
Avg. Trade [%]                    

Check`stats['_strategy']`

In [94]:
stats._strategy

<Strategy AboveSma(n1=20,n2=120)>

In [95]:
bt.plot(plot_volume=False, plot_pl=False)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p4206', ...)

Strategy optimization managed to up its initial performance _on in-sample data_ by almost 50% and even beat simple
[buy & hold](https://en.wikipedia.org/wiki/Buy_and_hold).
In real life optimization, however, do **take steps to avoid
[overfitting](https://en.wikipedia.org/wiki/Overfitting)**.

## Trade data

In addition to backtest statistics returned by
[`Backtest.run()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest.run)
shown above, you can look into _individual trade returns_ and the changing _equity curve_ and _drawdown_ by inspecting the last few, internal keys in the result series.

In [111]:
stats.tail()

Expectancy [%]                                            16.073535
SQN                                                        1.154652
_strategy                                                  AboveSma
_equity_curve                       Equity  DrawdownPct Drawdown...
_trades               Size  EntryBar  ExitBar  EntryPrice   Exit...
dtype: object

The columns should be self-explanatory.

In [112]:
stats['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.

,Equity,DrawdownPct,DrawdownDuration
2014-10-31,10000.000000,0.000000,NaT
2014-11-03,10000.000000,0.000000,NaT
2014-11-04,10000.000000,0.000000,NaT
2014-11-05,10000.000000,0.000000,NaT
2014-11-06,10000.000000,0.000000,NaT
...,...,...,...
2024-04-16,51168.879370,0.294405,NaT
2024-04-17,50430.518118,0.304587,NaT
2024-04-18,50301.597403,0.306364,NaT
2024-04-19,49235.077581,0.321071,NaT


In [113]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,474,280,282,21.059999,20.750000,-146.939747,-0.014720,2015-12-11,2015-12-15,4 days
1,513,418,509,19.170000,19.809999,328.319687,0.033385,2016-06-30,2016-11-08,131 days
2,513,509,528,19.809999,20.087000,142.101219,0.013983,2016-11-08,2016-12-06,28 days
3,513,528,547,20.087000,20.754999,342.683624,0.033255,2016-12-06,2017-01-04,29 days
4,513,547,997,20.754999,43.889999,11868.255117,1.114671,2017-01-04,2018-10-17,651 days
5,513,997,1031,43.889999,42.130001,-902.879139,-0.040100,2018-10-17,2018-12-06,50 days
6,480,1163,1202,44.980000,44.230000,-360.000000,-0.016674,2019-06-18,2019-08-13,56 days
7,479,1203,1204,44.320000,43.889999,-205.970146,-0.009702,2019-08-14,2019-08-15,1 days
8,463,1206,1666,45.450001,114.900002,32155.350353,1.528053,2019-08-19,2021-06-16,667 days
9,451,1668,1690,117.769997,120.875000,1400.356514,0.026365,2021-06-18,2021-07-21,33 days


Learn more by exploring further
[examples](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#tutorials)
or find more framework options in the
[full API reference](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#header-submodules).